In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install huggingface_hub transformers datasets gradio pipreqs TextBlob emot xformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) .

In [ ]:
!pip install accelerate>=0.20.1
!pip install transformers[torch] accelerate -U

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

In [ ]:
# Import libraries
import os
import uuid
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from scipy.special import softmax
import gradio as gr

from google.colab import drive
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import torch
from transformers import AutoTokenizer
from transformers import AutoConfig
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import IntervalStrategy
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback
from transformers import pipeline
from transformers import TrainingArguments
from transformers import Trainer
from torch import nn




In [ ]:
# Disabe W&B
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# Load the dataset and display some values

# Load the CSV file into a DataFrame

url = "https://github.com/Azubi-Africa/Career_Accelerator_P5-NLP/raw/master/zindi_challenge/data/Train.csv"

df = pd.read_csv(url)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   10001 non-null  object 
 1   safe_text  10001 non-null  object 
 2   label      10000 non-null  float64
 3   agreement  9999 non-null   float64
dtypes: float64(2), object(2)
memory usage: 312.7+ KB


In [ ]:
# Select rows with missing values
df.isnull().sum()

tweet_id     0
safe_text    0
label        1
agreement    2
dtype: int64

In [ ]:
# Select rows with missing values
df[df.isnull().any(axis=1)]

,tweet_id,safe_text,label,agreement
4798,RQMQ0L2A,#lawandorderSVU,NaN,NaN
4799,I cannot believe in this day and age some pare...,1,0.666667,NaN


In [ ]:
# Extract complete text from 'safe_text' column
complete_text = df.iloc[4798]['safe_text']
complete_text

'#lawandorderSVU '

In [ ]:
# Select row by index and assign values to columns
df.loc[4798, 'label'] = 0
df.loc[4798, 'agreement'] = 0.666667

# Use .iloc[] and .iat[] to select and update safe_text column
df.iloc[4798, df.columns.get_loc('safe_text')] = complete_text


In [ ]:
# Generate random UUID string for tweet_id
'''UUIDs are often used in software applications for various purposes such as generating unique IDs for entities,
tracking unique user sessions, or creating unique file names'''
rand_tweet_id = str(uuid.uuid4())

# Select row by index and assign values to columns
row_index = 4799
df.loc[row_index, 'tweet_id'] = rand_tweet_id
df.loc[row_index, 'label'] = 1
df.loc[row_index, 'agreement'] = 0.666667

# Use .iloc[] and .iat[] to select and update safe_text column
df.iloc[row_index, df.columns.get_loc('safe_text')] = df.iloc[row_index, 1]


In [ ]:
df[df.duplicated()].sum()

tweet_id     0.0
safe_text    0.0
label        0.0
agreement    0.0
dtype: float64

In [ ]:
df['label'].value_counts()


 0.0    4909
 1.0    4054
-1.0    1038
Name: label, dtype: int64

In [ ]:
# Find the maximum count among all classes
max_class_count = df['label'].value_counts().max()

# Group the dataframe by 'label'
grouped = df.groupby('label')

# Sample each group to match the max_class_count
balanced_df = grouped.apply(lambda x: x.sample(max_class_count, replace=True)).reset_index(drop=True)

# The 'balanced_df' now contains an equal number of instances for each class
print(balanced_df['label'].value_counts())


-1.0    4909
 0.0    4909
 1.0    4909
Name: label, dtype: int64


In [ ]:
# Split the train data => {train, eval}
train, eval = train_test_split(balanced_df, test_size=0.2, random_state=42)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11781 entries, 2435 to 7270
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   11781 non-null  object 
 1   safe_text  11781 non-null  object 
 2   label      11781 non-null  float64
 3   agreement  11781 non-null  float64
dtypes: float64(2), object(2)
memory usage: 460.2+ KB


In [ ]:
eval.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2946 entries, 13730 to 9863
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   2946 non-null   object 
 1   safe_text  2946 non-null   object 
 2   label      2946 non-null   float64
 3   agreement  2946 non-null   float64
dtypes: float64(2), object(2)
memory usage: 115.1+ KB


In [ ]:
# Save splitted subsets

# Define file path

file_path = '/content/drive/MyDrive/NLP/NLP_2/M1'

#"/content/drive/MyDrive/NLP/NLP_2/M1"

train.to_csv(os.path.join(file_path, "train_subset.csv"), index=False)
eval.to_csv(os.path.join(file_path, "eval_subset.csv"), index=False)

In [ ]:
# Load the CSV files into a dataset

dataset = load_dataset('csv', data_files={
    'train': '/content/drive/MyDrive/NLP/NLP_2/M1/train_subset.csv',
    'eval': '/content/drive/MyDrive/NLP/NLP_2/M1/eval_subset.csv'
}, encoding='ISO-8859-1')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f9c270008c929a4f/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',                          # Directory where the model checkpoints and evaluation results will be stored
    evaluation_strategy=IntervalStrategy.STEPS,      # Interval for evaluating the model during training (every specified number of steps)
    save_strategy=IntervalStrategy.STEPS,            # Interval for saving the model during training (every specified number of steps)
    save_steps=500,                                  # Number of steps between two saves
    load_best_model_at_end=True,                     # Whether to load the best model at the end of training
    num_train_epochs=16,                             # Number of training epochs
    per_device_train_batch_size=4,                   # Batch size per GPU for training
    per_device_eval_batch_size=4,                    # Batch size per GPU for evaluation
    learning_rate=3e-5,                              # Learning rate
    weight_decay=0.01,                               # Weight decay
    warmup_steps=500,                                # Number of warmup steps
    logging_steps=500,                               # Number of steps between two logs
    fp16=True,                                       # Whether to use 16-bit precision
    gradient_accumulation_steps=16,                  # Number of steps to accumulate gradients before performing an optimizer step
    dataloader_num_workers=2,                        # Number of workers to use for loading data
    push_to_hub=True,                                # Whether to push the model checkpoints to the Hugging Face hub
    hub_model_id="slickdata/finetuned-Sentiment-classfication-BERT-model",  # Model ID to use when pushing the model to the Hugging Face hub
)

#use hub_model_id="finetuned-Sentiment-classfication-ROBERTA-model
#use hub_model_id="finetuned-Sentiment-classfication-BERT-model
#use hub_model_id="finetuned-Sentiment-classfication-DISTILBERT-model
#use hub_model_id="finetuned-Sentiment-classfication-XLM-model

# Define the early stopping callback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,                       # Number of epochs with no improvement before stopping training
    early_stopping_threshold=0.01,                   # Minimum improvement in the metric for considering an improvement
)

# Combine the training arguments and the early stopping callback
training_args.callbacks = [early_stopping]


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
tokenizer_BERT = AutoTokenizer.from_pretrained('bert-base-cased')

'''
This code instantiates a tokenizer for the BERT (Bidirectional Encoder Representations from Transformers)
pre-trained model with the bert-base-cased configuration.

'''


'\nThis code instantiates a tokenizer for the BERT (Bidirectional Encoder Representations from Transformers)\npre-trained model with the bert-base-cased configuration.\n\n'

In [ ]:
# Define a function to transform the label values
def transform_labels(label):
    # Extract the label value
    label = label['label']
    # Map the label value to an integer value
    num = 0
    if label == -1: #'Negative'
        num = 0
    elif label == 0: #'Neutral'
        num = 1
    elif label == 1: #'Positive'
        num = 2
    # Return a dictionary with a single key-value pair
    return {'labels': num}

# Define a function to tokenize the text data
def tokenize_data(example):
    # Extract the 'safe_text' value from the input example and tokenize it
    return tokenizer_BERT(example['safe_text'], padding='max_length')

# Apply the transformation functions to the dataset using the 'map' method
# This transforms the label values and tokenizes the text data
dataset_out = dataset.map(transform_labels)

dataset_BERT = dataset_out.map(tokenize_data, batched=True)

# Define a list of column names to remove from the dataset
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']

# Apply the 'transform_labels' function to the dataset to transform the label values
# Also remove the columns specified in 'remove_columns'

dataset_BERT = dataset_BERT.map(transform_labels, remove_columns=remove_columns)

Map:   0%|          | 0/11781 [00:00<?, ? examples/s]

Map:   0%|          | 0/2946 [00:00<?, ? examples/s]

Map:   0%|          | 0/11781 [00:00<?, ? examples/s]

Map:   0%|          | 0/2946 [00:00<?, ? examples/s]

Map:   0%|          | 0/11781 [00:00<?, ? examples/s]

Map:   0%|          | 0/2946 [00:00<?, ? examples/s]

In [ ]:
# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model_BERT = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_dataset_BERT = dataset_BERT['train'].shuffle(seed=10) #.select(range(40000)) # to select a part


In [ ]:
eval_dataset_BERT = dataset_BERT['eval'].shuffle(seed=10)


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    rmse = np.sqrt(np.mean((predictions - labels)**2))
    return {"rmse": rmse}


In [ ]:
trainer_BERT = Trainer(
    model=model_BERT,
    args=training_args,
    train_dataset=train_dataset_BERT,
    eval_dataset=eval_dataset_BERT,
    compute_metrics=compute_metrics    # Add this line to define the compute_metrics function
)

Cloning https://huggingface.co/slickdata/finetuned-Sentiment-classfication-BERT-model into local empty directory.


Download file pytorch_model.bin:   0%|          | 18.4k/413M [00:00<?, ?B/s]

Download file runs/Jul15_12-23-59_ea8600d35e6e/events.out.tfevents.1689426105.ea8600d35e6e.384.1: 100%|#######…

Download file training_args.bin: 100%|##########| 4.12k/4.12k [00:00<?, ?B/s]

Download file runs/Jul15_12-23-59_ea8600d35e6e/events.out.tfevents.1689423873.ea8600d35e6e.384.0: 100%|#######…

...

Download file runs/Jul16_02-37-05_fb7708137902/events.out.tfevents.1689477401.fb7708137902.224.1: 100%|#######…

Clean file runs/Jul16_02-37-05_fb7708137902/events.out.tfevents.1689477401.fb7708137902.224.1: 100%|##########…

Download file runs/Jul22_20-32-18_7cec7019d63b/events.out.tfevents.1690061228.7cec7019d63b.409.1: 100%|#######…

Clean file runs/Jul22_20-32-18_7cec7019d63b/events.out.tfevents.1690061228.7cec7019d63b.409.1: 100%|##########…

Clean file pytorch_model.bin:   0%|          | 1.00k/413M [00:00<?, ?B/s]

In [ ]:
trainer_BERT.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Rmse
500,0.727900,0.368373,0.468636
1000,0.164200,0.318365,0.370776
1500,0.048400,0.347817,0.335701
2000,0.020300,0.404769,0.331120
2500,0.011200,0.405531,0.326993


TrainOutput(global_step=2944, training_loss=0.1664376830601174, metrics={'train_runtime': 7708.5133, 'train_samples_per_second': 24.453, 'train_steps_per_second': 0.382, 'total_flos': 4.956293761084109e+16, 'train_loss': 0.1664376830601174, 'epoch': 15.99})

In [ ]:
# Evaluate the model
eval_results = trainer_BERT.evaluate()

# Create a dictionary of the evaluation results
results_dict = {
    "Model": "Bert_base",
    "Loss": eval_results["eval_loss"],
    "RMSE": eval_results["eval_rmse"],
    "Runtime": eval_results["eval_runtime"],
    "Samples Per Second": eval_results["eval_samples_per_second"],
    "Steps Per Second": eval_results["eval_steps_per_second"],
    "Epoch": eval_results["epoch"]
}

# Create a pandas DataFrame from the dictionary
results_df = pd.DataFrame([results_dict])

# Sort the results by "eval_rmse" in ascending order and get the name and state dict of the best model
best_model = results_df.loc[results_df['RMSE'].idxmin()]

print(best_model)


Model                 Bert_base
Loss                   0.318365
RMSE                   0.370776
Runtime                 40.0499
Samples Per Second       73.558
Steps Per Second         18.402
Epoch                     15.99
Name: 0, dtype: object


In [ ]:
 # Push the final fine-tuned model to the Hugging Face model hub

trainer_BERT.push_to_hub("slickdata/finetuned-Sentiment-classfication-BERT-model")

Upload file runs/Jul22_21-47-18_272903a66f85/events.out.tfevents.1690062573.272903a66f85.496.0:   0%|         …

Upload file runs/Jul22_21-47-18_272903a66f85/events.out.tfevents.1690070322.272903a66f85.496.1:   0%|         …

To https://huggingface.co/slickdata/finetuned-Sentiment-classfication-BERT-model
   063d2b0..8578b90  main -> main

   063d2b0..8578b90  main -> main

To https://huggingface.co/slickdata/finetuned-Sentiment-classfication-BERT-model
   8578b90..4ea481b  main -> main

   8578b90..4ea481b  main -> main



'https://huggingface.co/slickdata/finetuned-Sentiment-classfication-BERT-model/commit/8578b9010bfa9e74b424909bc78521788abb44c7'

In [ ]:
tokenizer_BERT.push_to_hub("slickdata/finetuned-Sentiment-classfication-BERT-model")

CommitInfo(commit_url='https://huggingface.co/slickdata/finetuned-Sentiment-classfication-BERT-model/commit/6a680f05567d917f68b1b9ef38ec2fa1391bfab2', commit_message='Upload tokenizer', commit_description='', oid='6a680f05567d917f68b1b9ef38ec2fa1391bfab2', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model_BERT.push_to_hub("slickdata/finetuned-Sentiment-classfication-BERT-model")

CommitInfo(commit_url='https://huggingface.co/slickdata/finetuned-Sentiment-classfication-BERT-model/commit/75d95db4c487e6169f78ee2c983b2c02996eec52', commit_message='Upload BertForSequenceClassification', commit_description='', oid='75d95db4c487e6169f78ee2c983b2c02996eec52', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Load the tokenizer
tokenizer = tokenizer_BERT.from_pretrained("slickdata/finetuned-Sentiment-classfication-BERT-model")

# Load the fine-tuned model
model = pipeline("text-classification", model="slickdata/finetuned-Sentiment-classfication-BERT-model", tokenizer=tokenizer)


In [ ]:
label_map = {0: "negative", 1: "neutral", 2: "positive"}

# Make predictions on some example text
result = model("I love these covid vaccines.")

# Map the numerical label to the corresponding class name
result[0]["label"] = label_map[int(result[0]["label"].split("_")[1])]

# Print the predicted label and score
print(result)

[{'label': 'positive', 'score': 0.9705061912536621}]


In [ ]:
!pip freeze > /content/drive/MyDrive/NLP/NLP2/M1/requirements.txt


In [ ]:
!ls {file_path}

ls: cannot access '{file_path}': No such file or directory
